In [1]:
# allows editing aoc_utils "live" without restarting kernel
# see https://ipython.org/ipython-doc/stable/config/extensions/autoreload.html
# and https://stackoverflow.com/a/17551284
%load_ext autoreload
%autoreload 2

# Add the aoc_utils path
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import aoc_utils
get_input = aoc_utils.get_input
print = aoc_utils.debug_print

timer = aoc_utils.start_timer()

In [2]:
# Useful imports
import re
from collections import defaultdict, deque
import heapq
import functools
import queue
import itertools
import math
import random
from collections import Counter
import parse
import statistics

In [8]:
# aliases from utils
getnums = aoc_utils.getnums

In [9]:
def Day1(data=get_input(1, 2021)):
  nums = list(map(int, data))
  def p1():
    return len([i for i in range(1, len(nums)) if nums[i] > nums[i-1]])
  def p2():
    sums = [sum(nums[i:i+3]) for i in range(len(nums) - 2)]
    return len([i for i in range(1, len(sums)) if sums[i] > sums[i-1]])
    
  return p1(),p2()

assert Day1() == (1548, 1589)

In [10]:
def Day2(data=get_input(2,2021)):
  def p1():
    h,d=0,0
    for line in data:
      n = getnums(line)[0]
      if 'forward' in line:
        h += n
      elif 'down' in line:
        d += n
      else:
        d -= n
    return d * h
  def p2():
    h,d,aim=0,0,0
    for line in data:
      n = getnums(line)[0]
      if 'forward' in line:
        h += n
        d += aim * n
      elif 'down' in line:
        aim += n
      else:
        aim -= n
    return d * h
  return p1(),p2()

assert Day2() == (1694130, 1698850445),Day2()

In [13]:
def Day3(data = get_input(3,2021)):
  b2d = aoc_utils.bin_to_decimal

  def p1():
    cnts = [Counter() for _ in range(12)]
    for line in data:
      for idx,c in enumerate(line):
        cnts[idx].update(c)
    a = b2d(''.join([cnt.most_common()[0][0] for cnt in cnts]))
    b = b2d(''.join([cnt.most_common()[-1][0] for cnt in cnts]))
    return a*b

  def p2():
    def winnow(arr, most_com=True):
      for idx in range(len(arr[0])):
        cnt = Counter()
        for l in arr:
          cnt.update(l[idx])
        v = None
        if most_com:
          v = '0' if cnt['0'] > cnt['1'] else '1'
        else:
          v = '1' if cnt['1'] < cnt['0'] else '0'
        arr = [l for l in arr if l[idx] == v]
        if len(arr) == 1:
          return arr[0]

    O = winnow(data, most_com=True)
    C  = winnow(data, most_com=False)
    return b2d(O)*b2d(C)
  return p1(),p2()

assert Day3() == (1540244, 4203981)

In [14]:
def Day4(data=get_input(4,2021)):
  nums = getnums(data[0])
  boards = list(aoc_utils.split_list(data[2:]))
  grids = []
  for board in boards:
    grid = {}
    for y,line in enumerate(board):
      for x,num in enumerate(getnums(line)):
        grid[(x,y)] = num
    grids.append(grid)

  def boardsum(g, seen):
    unmarked = []
    for x in range(5):
      for y in range(5):
        n = g[(x,y)]
        if n not in seen:
          unmarked.append(n)
    return sum(unmarked)

  def winner(grid, seen):
    for x in range(5):
      if all(grid[(x,y)] in seen for y in range(5)):
        return True
    for y in range(5):
      if all(grid[(x,y)] in seen for x in range(5)):
        return True
    return False

  def p1():
    seen = set()
    for n in nums:
      seen.add(n)
      for g in grids:
        if winner(g, seen):
          return boardsum(g,seen)*n
  def p2():
    seen = set()
    winners = []
    remaining = grids
    for n in nums:
      seen.add(n)
      for g in remaining:
        if winner(g, seen):
          winners.append(g)
        if len(winners) == len(grids):
          return boardsum(g,seen)*n
      remaining = [_g for _g in remaining if _g not in winners]
  return p1(),p2()
assert Day4() == (71708, 34726)

In [15]:
def Day5(data=get_input(5,2021)):
  lines = list(map(getnums, data))
  def points(x1,y1,x2,y2):
    if y1 == y2:
      x1,x2 = sorted([x1,x2])
      return [(x,y1) for x in range(x1,x2+1)]
    elif x1 == x2:
      y1,y2 = sorted([y1,y2])
      return [(x1,y) for y in range(y1,y2+1)]
    else:
      m,b = statistics.linear_regression([x1,x2],[y1,y2])
      assert m == 1 or m == -1
      x1,x2 = sorted([x1,x2])
      return [(x,int(m*x+b)) for x in range(x1,x2+1)]

  assert points(0,0,2,0) == [(0,0), (1,0), (2,0)]
  assert points(0,0,0,2) == [(0,0), (0,1), (0,2)]
  assert points(2,0,0,0) == [(0,0), (1,0), (2,0)]
  assert points(0,2,0,0) == [(0,0), (0,1), (0,2)]
  assert points(1,1,1,3) == [(1,1),(1,2),(1,3)]
  assert sorted(points(9,7,7,7)) == [(7,7),(8,7),(9,7)]
  assert sorted(points(7,7,9,7)) == [(7,7),(8,7),(9,7)]
  assert sorted(points(0,0,3,3)) == [(0,0),(1,1),(2,2),(3,3)]

  p1_cnt = defaultdict(int)
  p2_cnt = defaultdict(int)
  for line in lines:
    x1,y1,x2,y2 = line
    for p in points(x1,y1,x2,y2):
      p2_cnt[p] += 1
      if x1 == x2 or y1 == y2:
        p1_cnt[p] += 1
  p1 = sum([1 for v in p1_cnt.values() if v > 1])
  p2 = sum([1 for v in p2_cnt.values() if v > 1])

  return p1,p2

assert Day5() == (5608, 20299)


In [17]:
def Day6(data = get_input(6, 2021)):
  p1 = 0
  p2 = 0
  data = getnums(data[0])
  P1_DAYS = 80
  P2_DAYS = 256
  cnt = Counter(data)
  for d in range(P2_DAYS):
    cyc = d%7
    tmp = cnt[7]
    cnt[7] = cnt[8]
    cnt[8] = cnt[cyc]
    cnt[cyc] += tmp
    if d + 1 == P1_DAYS:
      p1 = cnt.total()
  p2 = cnt.total()
  return p1,p2

assert Day6() == (360610, 1631629590423) 